In [1]:
import boto3
import gzip
import re
import requests
import shutil
import os
from zipfile import ZipFile

In [ ]:
# Test ZIP file download with unrestricted file
# This works
url = 'https://www.stats.govt.nz/assets/Uploads/Business-financial-data/Business-financial-data-December-2023-quarter/Download-data/business-financial-data-december-2023-quarter.zip'
r = requests.get(url)
with open('test.zip', 'wb') as f:
    f.write(r.content)

with ZipFile('test.zip', 'r') as f:
    f.extractall()

In [ ]:
# For now: Download restricted ZIP file manually
# TODO: Figure out how to login and download restricted files programmatically 

In [6]:
# Unzip 2023_CORE.zip
with ZipFile('./xml/2023_CORE.zip', 'r') as f:
    f.extractall('./xml/2023_CORE')

In [7]:
files = ['WR_2023_20240112181857_CORE_0002.xml.gz'] #, 'WR_2023_20240112181857_CORE_0002.xml.gz']
s3r = boto3.resource('s3')

In [14]:
# Toy write-and-delete workflow
file = 'test.txt'
f = open(file, 'w')
f.write('Testing')
f = open(file, 'rb')
s3r.Bucket('wos-bucket').put_object(
          Key = 'test', # File name
          Body = f) # File
os.remove(file)

In [3]:
for file in files:
    gz = './xml/2023_CORE/' + file
    xml = './xml/2023_CORE/' + file[:-3]
    with gzip.open(gz, 'rb') as zipped:
       with open(xml, 'wb') as unzipped:
         shutil.copyfileobj(zipped, unzipped)
    body = open(xml[6:], 'rb')
    s3r.Bucket('wos-bucket').put_object(
          Key = xml, # File name
          Body = body) # File
    # Delete files to make space
    os.remove(gz)
    os.remove(xml)
    

FileNotFoundError: [Errno 2] No such file or directory: '~/.TrashWR_2023_20240112181857_CORE_0001.xml.gz'

In [27]:
# Unzip .gz files in 2023_CORE
# Adapted from https://bito.ai/resources/unzip-gz-file-python-python-explained/

for file in os.listdir('./xml/2023_CORE'):
  if file.endswith('.gz'): 
     with gzip.open('./xml/2023_CORE/' + file, 'rb') as f_in:
       with open('./xml/2023_CORE/' + file[:-3], 'wb') as f_out:
         shutil.copyfileobj(f_in, f_out)
         xml = open(f_out)

OSError: [Errno 28] No space left on device

In [3]:
# Create S3 bucket
s3c = boto3.client('s3', region_name = 'us-east-1')
bucket = s3c.create_bucket(Bucket = 'wos-bucket')

In [5]:
# For now: Write XML to S3
s3r = boto3.resource('s3')
s3r.Bucket('wos-bucket').put_object(
                Key = 'xml/2023_CORE/WR_2023_20240112181857_CORE_0001.xml.gz', # File name
                Body = xml) # XML file

s3.Object(bucket_name='wos-bucket', key='xml/2023_CORE/WR_2023_20240112181857_CORE_0001.xml.gz')